### 本测试是为了测试Llama2-7b的Tokenizer是否会自动加入 EOS

In [3]:
from typing import Dict, Optional, Sequence
import torch
import transformers
import utils
from torch.utils.data import Dataset
from transformers import Trainer

In [4]:
# 定义一个常量IGNORE_INDEX,其值为-100。
# 在计算损失函数时,这个值表示需要忽略对应位置的标签。例如,在进行SFT任务时,输入序列部分的标签通常会被标记为IGNORE_INDEX,以避免对输入序列的token进行惩罚。
# 而定义为 -100 是为了契合transformers, transformers库默认 -100 为忽略loss计算的标签
IGNORE_INDEX = -100
# 定义了一些默认的特殊标记(token),如填充标记(PAD)、结束标记(EOS)、开始标记(BOS)和未知标记(UNK)。
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"

In [10]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    # 2. 它接受两个参数:strings(字符串序列)和tokenizer(tokenizer实例)。
    # 3. 函数使用tokenizer的__call__方法对每个字符串进行tokenize,并将结果存储在tokenized_list中。
    """Tokenize a list of strings."""
    # 对每个字符串进行tokenize,并将结果存储在tokenized_list中, padding="longest"表示将序列填充到最长长度
    # max_length指定最大序列长度,超过该长度的部分将被截断, truncation=True表示允许截断
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    # 从tokenize结果中提取input_ids和labels
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    # 计算每个序列的实际长度,不包括填充标记
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


def smart_tokenizer_and_embedding_resize(special_tokens_dict: Dict,
                                         tokenizer: transformers.PreTrainedTokenizer):
    # 1. 这是一个函数,用于调整tokenizer和embedding的大小。
    # 2. 它接受三个参数:special_tokens_dict(特殊标记字典)、tokenizer(tokenizer实例)和model(模型实例)。
    """Resize tokenizer and embedding.
    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    # 3. 首先,它使用tokenizer.add_special_tokens方法向tokenizer添加特殊标记,并获取新增标记的数量。
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    return num_new_tokens


def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    # 2. 它接受两个参数:strings(字符串序列)和tokenizer(tokenizer实例)。
    # 3. 函数使用tokenizer的__call__方法对每个字符串进行tokenize,并将结果存储在tokenized_list中。
    """Tokenize a list of strings."""
    # 对每个字符串进行tokenize,并将结果存储在tokenized_list中, padding="longest"表示将序列填充到最长长度
    # max_length指定最大序列长度,超过该长度的部分将被截断, truncation=True表示允许截断
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    # 从tokenize结果中提取input_ids和labels
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    # 计算每个序列的实际长度,不包括填充标记
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [11]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "Llama2-7b-hf-tokenizer/Llama-2-7b-hf",
        cache_dir=None,
        model_max_length=512,
        padding_side="right"
    )
special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN
tokenizer

LlamaTokenizerFast(name_or_path='Llama2-7b-hf-tokenizer/Llama-2-7b-hf', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [12]:
print(smart_tokenizer_and_embedding_resize(special_tokens_dict, tokenizer))
# 仅仅添加了 '[PAD]' , 因为Llama的 tokenizer 中没有pad
tokenizer

1


LlamaTokenizerFast(name_or_path='Llama2-7b-hf-tokenizer/Llama-2-7b-hf', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [17]:
texts = ["I love you","Nice to meet you"]
res = _tokenize_fn(texts, tokenizer)
res

{'input_ids': [tensor([   1,  306, 5360,  366]),
  tensor([    1, 20103,   304,  5870,   366])],
 'labels': [tensor([   1,  306, 5360,  366]),
  tensor([    1, 20103,   304,  5870,   366])],
 'input_ids_lens': [4, 5],
 'labels_lens': [4, 5]}

In [19]:
res["input_ids"]

[tensor([   1,  306, 5360,  366]), tensor([    1, 20103,   304,  5870,   366])]

### 结论: Llama2-7b的Tokenizer是不会自动在句子末尾加入 EOS符号的
#### 我们可以手动为其加入,